In [68]:
def load(file_name):
    # store the gene expression file
    file = open(file_name + ".txt", "r")
    data = [line.strip().split('\t') for line in file.readlines()]
    for x in range(len(data)):
        flag=1
        for y in range(len(data[x])):
                data[x][y] = 'G' + str(flag) + '_' + data[x][y]
                flag+=1                
    
    return data

data = load(r"C:\Users\17169\Documents\association-rule-test-data")

In [69]:
def get_support_count(input_list, dataset_list):
    count = 0
    for data_set in dataset_list:
        if set(input_list).issubset(data_set):
            count = count + 1
    return count


In [70]:
def generate_sets(size_of_set, previous_freq_dataset_list, dataset_list, support_count):
    i = 0
    list_of_sets = []
    flag = False
    if size_of_set == 1:
        dic={}
        for sample in dataset_list:
            for gene in sample:
                if gene in dic:
                    dic.update({gene:dic.get(gene) +1})
                else: 
                    dic.update({gene:1})
        for key in list(dic.keys()):
            if dic.get(key) < support_count:
                del dic[key]      
        return list(dic)
    else:
        for first_dataset in previous_freq_dataset_list[i:]:
            for second_dataset in previous_freq_dataset_list[i+1:]:
                if size_of_set == 2:
                    pair = []
                    pair.append(first_dataset)
                    pair.append(second_dataset)
                    if get_support_count(pair, dataset_list) >= support_count:
                        list_of_sets.append(pair)
                else:
                    for ele in range(size_of_set - 2):
                        if first_dataset[ele] != second_dataset[ele]:
                            flag = True
                            break
                    if flag:
                        flag = False
                        break
                    else:
                        dataset = []
                        dataset = dataset+first_dataset
                        dataset.append(second_dataset[len(second_dataset) - 1])
                        sc = get_support_count(dataset, dataset_list)
                        if sc >= support_count:
                            list_of_sets.append(dataset)
            i = i + 1
    return list_of_sets 


In [71]:
def calulate_frequent_set_count(support_count_percent, dataset_list):
    frequency_dic = {}
    support_count = int(support_count_percent*len(dataset_list)/100)
    freq_set_length_count = 1
    previous_freq_dataset_list = []
    set_size = 1
    while freq_set_length_count > 0:
        previous_freq_dataset_list = generate_sets(set_size, previous_freq_dataset_list, dataset_list, support_count)
        freq_set_length_count = len(previous_freq_dataset_list)
        if freq_set_length_count > 0:
            frequency_dic.update({set_size : freq_set_length_count})
        set_size += 1
    return frequency_dic

In [72]:
length_dic = calulate_frequent_set_count(support, data)

In [73]:
for length in length_dic:
		print("number of length",length ,"frequent itemsets:", length_dic[length])

number of length 1 frequent itemsets: 196
number of length 2 frequent itemsets: 5340
number of length 3 frequent itemsets: 5287
number of length 4 frequent itemsets: 1518
number of length 5 frequent itemsets: 438
number of length 6 frequent itemsets: 88
number of length 7 frequent itemsets: 11
number of length 8 frequent itemsets: 1


In [74]:
def frequentSetGeneration(data, min_sup = 50):
    min_sup /=100
    K = 1
    L = []
    result = dict()
    size = len(data)
    L, dictionary = generate1ItemSet(data, min_sup)
    result = dictionary
    dataSet = convertToSet(data)
    while True:
        K += 1
        l = set()
        for a in L:
            for b in L:
                tmp = a | b
                tmp = sorted(tmp)
                if len(tmp) == K:
                    l.add(frozenset(tmp))
        L = []
        for a in l:
            count = 0
            for i in dataSet:
                if a.issubset(i):
                    count+=1
            if count/size >= min_sup:
                L.append(a)
                result[a] = count
        if len(L) == 0:
            break
        else:
            printUtil(L,len(L),K)
    return result

In [75]:
class AssocRule:

    def __init__(self, min_sup, min_conf, filePath):
        self.min_sup = min_sup
        self.min_conf = min_conf
        self.freqItemsets = frequentSetGeneration(data, min_sup = self.min_sup)
        self.rules = []
        self.associationRules()        

    def associationRules(self):
        for freqItemset in self.freqItemsets:
            if len(freqItemset) > 1:
                self.findRules(freqItemset)

    def findRules(self, freqItemset):
        size = len(freqItemset)
        prev = []
        for item in freqItemset:
            tempItems = set(freqItemset)
            tempItems.remove(item)
            temp = [frozenset(tempItems), frozenset({item})]
            if self.checkMinConfidence(temp):
                self.rules.append(temp)
                prev.append(temp)
        
        while size > 2:
            temp = []
            combinations = itertools.combinations(prev, 2)
            for combination in combinations:
                intersection = combination[0][0] & combination[1][0]
                union = combination[0][1] | combination[1][1]
                if len(intersection) == size - 2:
                    tempList = [frozenset(set(intersection)), frozenset(set(union))]
                    if self.checkMinConfidence(tempList):
                        temp.append(tempList)

            prev = temp
            self.rules += temp

            size -= 1

    def checkMinConfidence(self, rule):
        union = rule[0] | rule[1]
        supCount = float(self.freqItemsets[union])
        if (supCount / float(self.freqItemsets[rule[0]])) >= self.min_conf:
            return True   
        return False

    def template1(self, cond, count, items):
        result = []
        for rule in self.rules:
            if self.checkTemplate1(rule, cond, count, items):
                result.append(rule)
        result = self.removeDuplicates(result)
        return result, len(result)

    def checkTemplate1(self, rule, cond, count, items):
        if cond == "RULE" and count == "ANY":
            if len((rule[0] | rule[1]) & set(items)) > 0:
                return True
        elif cond == "RULE" and count == "NONE":
            if len((rule[0] | rule[1]) & set(items)) == 0:
                return True
        elif cond == "RULE" and isinstance(count, int):
            if len((rule[0] | rule[1]) & set(items)) == count:
                return True
        elif cond == "HEAD" and count == "ANY":
            if len(rule[0] & set(items)) > 0:
                return True
        elif cond == "HEAD" and count == "NONE":
            if len(rule[0] & set(items)) == 0:
                return True
        elif cond == "HEAD" and isinstance(count, int):
            if len(rule[0] & set(items)) == count:
                return True
        elif cond == "BODY" and count == "ANY":
            if len(rule[1] & set(items)) > 0:
                return True
        elif cond == "BODY" and count == "NONE":
            if len(rule[1] & set(items)) == 0:
                return True
        elif cond == "BODY" and isinstance(count, int):
            if len(rule[1] & set(items)) == count:
                return True
        return False
        

    def template2(self, cond, count):
        result = []
        for rule in self.rules:
            if self.checkTemplate2(rule, cond, count):
                result.append(rule)
        result = self.removeDuplicates(result)
        return result, len(result)

    def checkTemplate2(self, rule, cond, count):
        if cond == "RULE" and (len(rule[0]) + len(rule[1])) >= count:
            return True
        elif cond == "HEAD" and len(rule[0]) >= count:
            return True
        elif cond == "BODY" and len(rule[1]) >= count:
            return True 
        return False

    def template3(self, *args):
        result = []
        for rule in self.rules:
            if self.checkTemplate3(rule, args):
                result.append(rule)
        result = self.removeDuplicates(result)
        return result, len(result)

    def checkTemplate3(self, rule, args):
        condition = args[0]
        if condition[0] == '1' and condition[-1] == '1':
            res1 = self.checkTemplate1(rule, args[1], args[2], args[3])
            res2 = self.checkTemplate1(rule, args[4], args[5], args[6])
        elif condition[0] == '1' and condition[-1] == '2':
            res1 = self.checkTemplate1(rule, args[1], args[2], args[3])
            res2 = self.checkTemplate2(rule, args[4], args[5])
        elif condition[0] == '2' and condition[-1] == '1':
            res1 = self.checkTemplate2(rule, args[1], args[2])
            res2 = self.checkTemplate1(rule, args[3], args[4], args[5])
        elif condition[0] == '2' and condition[-1] == '2':
            res1 = self.checkTemplate2(rule, args[1], args[2])
            res2 = self.checkTemplate2(rule, args[3], args[4])

        if condition[1:-1] == 'and':
            return res1 and res2
        elif condition[1:-1] == 'or':
            return res1 or res2

    def removeDuplicates(self, result):
        resultList = set(tuple(x) for x in result)
        return [list(x) for x in resultList]


In [76]:
(result11, cnt) = asso_rule.template1("RULE", "ANY", ['G59_Up'])

In [77]:
 asso_rule = AssocRule(float(min_sup), float(min_conf)/100, data)

Number of length 2 frequent itemsets 63
Number of length 3 frequent itemsets 2


In [78]:
min_sup = 50

In [79]:
min_conf = 70

In [80]:
import itertools

In [81]:
result11

[[frozenset({'G87_Up'}), frozenset({'G59_Up'})],
 [frozenset({'G59_Up'}), frozenset({'G82_Down'})],
 [frozenset({'G59_Up', 'G82_Down'}), frozenset({'G72_Up'})],
 [frozenset({'G59_Up', 'G96_Down'}), frozenset({'G72_Up'})],
 [frozenset({'G32_Down'}), frozenset({'G59_Up'})],
 [frozenset({'G96_Down'}), frozenset({'G59_Up', 'G72_Up'})],
 [frozenset({'G10_Down'}), frozenset({'G59_Up'})],
 [frozenset({'G82_Down'}), frozenset({'G59_Up', 'G72_Up'})],
 [frozenset({'G1_Up'}), frozenset({'G59_Up'})],
 [frozenset({'G72_Up'}), frozenset({'G59_Up', 'G82_Down'})],
 [frozenset({'G38_Down'}), frozenset({'G59_Up'})],
 [frozenset({'G72_Up'}), frozenset({'G59_Up'})],
 [frozenset({'G59_Up'}), frozenset({'G88_Down'})],
 [frozenset({'G59_Up', 'G72_Up'}), frozenset({'G96_Down'})],
 [frozenset({'G59_Up'}), frozenset({'G72_Up'})],
 [frozenset({'G72_Up', 'G82_Down'}), frozenset({'G59_Up'})],
 [frozenset({'G72_Up', 'G96_Down'}), frozenset({'G59_Up'})],
 [frozenset({'G6_Up'}), frozenset({'G59_Up'})],
 [frozenset({'

In [83]:
cnt

26